In [1]:
import numpy as np
import tensorflow as tf

In [2]:
import tensorflow_datasets as tfds

In [3]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# with_info=True provides tuple containing information about the version, features, number of samples. I used these info to store it in the mnist_info
# as_supervised=True loads the dataset as a 2-tuple structure (input, target) to separate them.

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
# Here I extracted the training and testing datasets with the built references


num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
# I defined the number of validation samples (10%) as a % of the training samples and cast the number as an integer.

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)
# I stored the number of test samples in a dedicated variable.

def scale(image, label):
    image = tf.cast(image, tf.float32)
    # The value must be a float since the possible values for the inputs are 0 to 255 (256 different shades of grey)
    image /= 255.
    # Returns a value between 0 and 1.
    return image, label
# The function 'scale' takes an MNIST image and its label. It prefers inputs between 0 and 1.

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)
# The method .map() applies a custom transformation to the given dataset.
# I scaled them all so they have the same magnitude.
# There was no need to shuffle the test data because I didn't be training my model using it.
# I decided that a single Batch would be equal to the size of the test data to hasten the process.

BUFFER_SIZE = 10000
# I can't shuffle the whole dataset in one go because it can't fit in memory, so I set this BUFFER_SIZE parameter for this huge dataset.

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
# I shuffled the train and validation data and specified the BUFFER_SIZE.

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
# My validation data was equal to 10% of the training set.
# Here I used the .take() method to take that many samples

train_data = shuffled_train_and_validation_data.skip(num_validation_samples)
# The train_data is everything else, so I skipped as many samples as there are in the validation dataset.

BATCH_SIZE = 150
# I determined the optimal batch size here.

train_data = train_data.batch(BATCH_SIZE)
# Here I took the advantage to batch the train data to be able to iterate over the different batches.

validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)
# I batched the train and test datasets here.

validation_inputs, validation_targets = next(iter(validation_data))

2022-03-06 19:17:40.238219: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-06 19:17:40.238642: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: No module named 'tensorflow_core.estimator'


    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: No module named 'tensorflow_core.estimator'


    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: No module named 'tensorflow_core.estimator'


    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: No module named 'tensorflow_core.estimator'


    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: No module named 'tensorflow_core.estimator'


    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: No module named 'tensorflow_core.estimator'


In [4]:
input_size = 784
output_size = 10
# Use same hidden layer size for both hidden layers. Not a necessity.
# There is a convenient method 'Flatten' that simply takes our 28x28x1 tensor and orders it into a (None,) or (28x28x1,) = (784,) vector
hidden_layer_size = 2500

model = tf.keras.Sequential([
       
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    # This is the first layer (the input layer)
    # each observation is 28x28x1 pixels, therefore it is a tensor of rank 3
    # I used the 'flatten' to actually create a feed forward neural network
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    # 2nd hidden layer
     tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    # 3rd hidden layer
     tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    # 4th hidden layer
     tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    # 5th hidden layer
    # This code outputs: activation(dot(input, weight) + bias)
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
    # the final layer is no different, except that I activated it using the Softmax method.
  
])

In [5]:
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [6]:
NUM_EPOCHS = 10
#
model.fit(train_data,
    epochs=NUM_EPOCHS,
    validation_data=(validation_inputs, validation_targets),
    validation_steps=10, verbose=2)

Epoch 1/10


2022-03-06 19:21:00.271568: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


360/360 - 198s - loss: 0.2648 - accuracy: 0.9227 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10


2022-03-06 19:24:19.831635: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


360/360 - 200s - loss: 0.1163 - accuracy: 0.9686 - val_loss: 0.0784 - val_accuracy: 0.9760
Epoch 3/10


...

360/360 - 210s - loss: 0.0387 - accuracy: 0.9895 - val_loss: 0.0495 - val_accuracy: 0.9885
Epoch 9/10


2022-03-06 19:49:26.114006: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


360/360 - 212s - loss: 0.0367 - accuracy: 0.9902 - val_loss: 0.0507 - val_accuracy: 0.9908
Epoch 10/10


2022-03-06 19:52:58.089380: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


360/360 - 212s - loss: 0.0426 - accuracy: 0.9896 - val_loss: 0.0317 - val_accuracy: 0.9918


In [7]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 9s 9s/step - loss: 0.1144 - accuracy: 0.9793


2022-03-06 19:53:10.483165: W tensorflow/core/common_runtime/base_collective_executor.cc:216] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
	 [[{{node IteratorGetNext}}]]


In [8]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))
# Better formatting

Test loss: 0.11. Test accuracy: 97.93%
